# RAG Usando el Servicio de Agente de Azure AI y Semantic Kernel

Este fragmento de código demuestra cómo crear y gestionar un agente de Azure AI para generación aumentada por recuperación (RAG) utilizando el `Azure AI Agent Service` y `Semantic Kernel`. El agente procesa las consultas de los usuarios basándose en el contexto recuperado y proporciona respuestas precisas en consecuencia.


Corrección de la versión de SQLite  
Si encuentras el error:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

Descomenta este bloque de código al inicio de tu notebook:  


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### Importando paquetes
El siguiente código importa los paquetes necesarios:


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# Generación Aumentada por Recuperación con Semantic Kernel y Azure AI Agent Service

Este ejemplo demuestra cómo utilizar el **Azure AI Agent Service** para realizar **Generación Aumentada por Recuperación (RAG)** combinando un modelo de lenguaje con contexto específico del dominio a partir de un documento cargado.

### Cómo Funciona

1. **Carga de Documento**: Se sube un archivo markdown (document.md) que contiene información (la póliza de seguro de viaje de Contoso) al servicio de agentes.

2. **Creación de Almacén Vectorial**: El documento se indexa en un almacén vectorial para habilitar la búsqueda semántica sobre su contenido.

3. **Configuración del Agente**: Se instancia un agente utilizando el modelo `gpt-4o` con las siguientes instrucciones estrictas:
   - Responder únicamente preguntas basadas en el contenido recuperado del documento.
   - Negarse a responder si la pregunta está fuera del alcance.

4. **Integración de la Herramienta de Búsqueda de Archivos**: Se registra la `FileSearchTool` con el agente, permitiendo que el modelo busque y recupere fragmentos relevantes del documento indexado durante la inferencia.

5. **Interacción con el Usuario**: Los usuarios pueden hacer preguntas. Si se encuentra información relevante en el documento, el agente genera una respuesta fundamentada.  
   Si no, el agente responde explícitamente que el documento no contiene información suficiente.


### Función Principal

La función principal es el punto de entrada de tu programa. Es donde comienza la ejecución del código y generalmente contiene la lógica principal.

[!NOTE] Asegúrate de que la función principal sea clara y fácil de entender.

#### Ejemplo de código

```python
def main():
    print("Hola, mundo!")
```

#### Comentarios

- Usa la función `main()` para organizar tu código.
- Llama a `main()` al final del archivo para iniciar la ejecución.

[!TIP] Es una buena práctica incluir una función principal en tus programas para mejorar la legibilidad y la estructura del código.

### Manejo de Errores

El manejo de errores es crucial para crear programas robustos. Asegúrate de anticipar posibles problemas y manejar las excepciones adecuadamente.

#### Ejemplo de código

```python
try:
    resultado = 10 / 0
except ZeroDivisionError:
    print("Error: División por cero.")
```

#### Comentarios

- Usa bloques `try` y `except` para capturar errores.
- Proporciona mensajes de error claros para ayudar a los usuarios a entender qué salió mal.

[!WARNING] No ignores los errores. Manejar las excepciones correctamente puede prevenir fallos inesperados en tu programa.

### Buenas Prácticas

Seguir buenas prácticas de programación puede hacer que tu código sea más eficiente y fácil de mantener.

#### Recomendaciones

- Escribe comentarios útiles y relevantes.
- Usa nombres de variables descriptivos.
- Divide tu código en funciones pequeñas y manejables.

[!IMPORTANT] Mantén tu código limpio y organizado para facilitar su comprensión y mantenimiento.

### Conclusión

Escribir código claro y manejable es esencial para cualquier desarrollador. Sigue las buenas prácticas y asegúrate de manejar los errores correctamente para crear programas confiables y fáciles de usar.


Asegúrate de ejecutar primero `az login` utilizando la CLI de Azure para que se proporcione el contexto de autenticación adecuado al usar `DefaultAzureCredential`. El servicio Azure AI Agent no utiliza claves de API.


In [ ]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        file: OpenAIFile = await client.agents.upload_file_and_poll(file_path="document.md", purpose="assistants")
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

Entendido, por favor proporciona el archivo Markdown que deseas traducir.



---

**Descargo de responsabilidad**:  
Este documento ha sido traducido utilizando el servicio de traducción automática [Co-op Translator](https://github.com/Azure/co-op-translator). Si bien nos esforzamos por garantizar la precisión, tenga en cuenta que las traducciones automatizadas pueden contener errores o imprecisiones. El documento original en su idioma nativo debe considerarse la fuente autorizada. Para información crítica, se recomienda una traducción profesional realizada por humanos. No nos hacemos responsables de malentendidos o interpretaciones erróneas que puedan surgir del uso de esta traducción.
